# Questions with Answers

### 1. Explain the difference between “__new__” and “__init__” in Python class instantiation, and provide a scenario where overriding new is necessary (e.g., for implementing a singleton pattern).

- `__new__` is a **static method** responsible for **creating a new instance** of a class. → useful for singleton for example.
- It is called **before** `__init__`.
- `__init__` is an **instance method** that initializes the newly created object.

In [1]:
# singleton

class Singleton:
	_instance = None
	def __new__(cls, *args, **kwargs):
		if cls._instance is None:
			cls._instance = super().__new__(cls)
		return cls._instance
		
	def __init__(self, value, *args, **kwargs):
		if not hasattr(self, "_initialized"):
			self.value = value
			self._initialized = True
			
			
# usage
s1 = Singleton(10)
s2 = Singleton(20)

print(s1.value)    
print(s2.value)    

10
10


---

### 2. What are metaclasses in Python? Write a custom metaclass that automatically registers all subclasses and enforces that every class has a specific method (e.g., validate()). Discuss real-world uses, such as in ORM frameworks like Django.

metaclass → classes of classes

- **a metaclass controls class creation and modification**.

1. **Instance** → an object of a class.
2. **Class** → defines how instances behave.
3. **Metaclass** → defines how classes behave.
- In Python, **everything is an object**.
- Classes themselves are **instances of a metaclass**.

• The default metaclass is `type`:

```python
class MyClass:
	pass
	
print(type(MyClass))   # <class 'type'>
```


**Key Idea:** Metaclasses let you **intercept class creation** and customize it.

A metaclass usually overrides **`__new__`** or **`__init__`**

In [2]:
class MyMeta(type):
	def __new__(cls, name, bases, namespace):
		print(f"Creating class {name}")
		return super().__new__(cls, name, bases, namespace)
		
class Person(metaclass=MyMeta):
	def greet(self):
		print("Hello")
		
# Output: Creating class Person -> when the class is defined (this is shown when we just define the class!)

Creating class Person


`MyMeta.__new__` is called **when the class is defined**, not when instances are created.

In [3]:
# another example of meta class
class RegistryMeta(type):
    registry = []

    def __new__(cls, name, bases, namespace):
        # Enforce 'validate' method
        if 'validate' not in namespace:
            raise TypeError(f"Class {name} must implement a 'validate()' method")
        
        # Create the class
        new_class = super().__new__(cls, name, bases, namespace)
        
        # Register the class
        cls.registry.append(new_class)
        return new_class

# Base class
class Base(metaclass=RegistryMeta):
    def validate(self):
        pass

# Subclasses
class User(Base):
    def validate(self):
        print("Validating user")

class Product(Base):
    def validate(self):
        print("Validating product")

print(RegistryMeta.registry)

[<class '__main__.Base'>, <class '__main__.User'>, <class '__main__.Product'>]


Singleton Pattern with metaclass:

In [4]:
class SingletonMeta(type):
    _instances = {}
    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super().__call__(*args, **kwargs)
        return cls._instances[cls]

class Logger(metaclass=SingletonMeta):
    pass

a = Logger()
b = Logger()
print(a is b)  # True

True


at first it is confusing. but just consider the order of execution.
Ultra-Short Rule (Memorize This)

class → metaclass `__new__()` → metaclass `__call__` → class `__new__` → class `__init__`

#### Real-World Uses of Metaclasses

1. ORM Frameworks (Django, SQLAlchemy)

- Django’s ModelBase metaclass scans model fields and prepares database mappings.
- You don’t have to write SQL manually; metaclass handles it.

2. Plugin Systems

- Automatically register subclasses, so plugins are discoverable without extra boilerplate.

3. Singleton Pattern (as i wrote above)

4. Enforcing Class Interfaces

Automatically ensure certain methods or attributes(like ORM fields) exist on all subclasses (like our validate() example).

---

### 3. Describe Python's Global Interpreter Lock (GIL). How does it impact multithreading, and when would you use multiprocessing instead of threading? Provide an example where the GIL causes unexpected performance issues.

GIL -> a mutex inside the CPython interpreter -> ensures only one thread executes Python bytecode at any given time.

what is mutex? -> mutual exclusion : a synchronization primitive -> used in concurrent programming -> ensure that only one thread or process at a time can access a shared resource or execute a critical section of code.

=> At any moment, at most one owner can hold a mutex.

In [5]:
import threading

lock = threading.Lock()   # mutex
counter = 0

def increment():
    global counter
    with lock:              # acquire
        counter += 1        # critical section
                            # release automatically

threads = [threading.Thread(target=increment) for _ in range(1000)]
for t in threads:
    t.start()
for t in threads:
    t.join()

print(counter)

1000


why GIL exisits? -> because CPython’s core data structures : are not thread-safe.

The GIL:

- Protects reference counting
- Avoids fine-grained locking across every object
- Simplifies the interpreter design
- Improves single-threaded performance

The GIL DOES lock: Execution of Python bytecode

the question is : why not use OS for managing threads? -> answer: 

The OS does not know the rules of Python objects:

- Python objects like lists, dicts, sets have internal structure

- Reference counts track how many references exist for memory management

- Operations like my_list.append(1) are not atomic — they involve multiple steps:

1. Check memory

2. Update internal array

3. Update reference counts

If two threads run these steps at the same time:

- OS does not know that Python needs these steps to be atomic

- Result: memory corruption, crashes, incorrect data






NEXT QUESTION: What would remove the GIL???

To remove the GIL, you need:

A different Python implementation that is thread-safe:

- Jython (runs on Java, no GIL)

- IronPython (runs on .NET, no GIL)

- Or rewrite CPython internals so that all objects are thread-safe (very hard, huge overhead)

#### when would you use multiprocessing instead of threading?

| Concept    | Threading                         | Multiprocessing                                        |
| ---------- | --------------------------------- | ------------------------------------------------------ |
| Uses       | Threads in **same process**       | Separate **processes**                                 |
| Memory     | Shared                            | Separate memory (do not share globals by default)      |
| GIL impact | Still limited by GIL for CPU code | Each process has **its own Python interpreter + GIL**  |
| Overhead   | Low                               | Higher (process creation, inter-process communication) |


Use threading if your program is I/O-bound, meaning it spends a lot of time waiting for:

- Network requests (API calls, sockets)

- Disk operations (file read/write)

- Database queries

In [6]:
import threading
import requests

urls = ["http://google.com"] * 10
def fetch(url):
    requests.get(url)
    
    
threads = [threading.Thread(target=fetch, args=(u,)) for u in urls]
[t.start() for t in threads]
[t.join() for t in threads]

[None, None, None, None, None, None, None, None, None, None]

In [7]:
threads

[<Thread(Thread-1004 (fetch), stopped 126369996592832)>,
 <Thread(Thread-1005 (fetch), stopped 126370004985536)>,
 <Thread(Thread-1006 (fetch), stopped 126370013378240)>,
 <Thread(Thread-1007 (fetch), stopped 126370021770944)>,
 <Thread(Thread-1008 (fetch), stopped 126378585028288)>,
 <Thread(Thread-1009 (fetch), stopped 126378576619200)>,
 <Thread(Thread-1010 (fetch), stopped 126378568210112)>,
 <Thread(Thread-1011 (fetch), stopped 126378559801024)>,
 <Thread(Thread-1012 (fetch), stopped 126378551391936)>,
 <Thread(Thread-1013 (fetch), stopped 126377802069696)>]

we used threads because it was i/o task.

Use multiprocessing if your program is CPU-bound, meaning it does heavy computations like:

- Number crunching

- Simulations

- Backtesting large datasets

In [8]:
from multiprocessing import Pool


def compute(n):
    return sum(i*i for i in range(n))

with Pool(processes=4) as pool:
    results = pool.map(compute, [10**7]*4)
    
results

[333333283333335000000,
 333333283333335000000,
 333333283333335000000,
 333333283333335000000]

- Each process has its own interpreter and GIL

- Computation runs in true parallel on multiple CPU cores

#### Provide an example where the GIL causes unexpected performance issues.

In [9]:
# Example: CPU-bound computation with threads
import threading
import time

# CPU-heavy task
def count(n):
    total = 0
    for i in range(n):
        total += i*i
    return total

N = 100_000_000  # large number for noticeable computation

# Using threads
threads = []
start = time.time()
for _ in range(4):  # 4 threads
    t = threading.Thread(target=count, args=(N,))
    threads.append(t)
    t.start()

for t in threads:
    t.join()

print("Threading took:", time.time() - start)

Threading took: 15.040069103240967


In [10]:
import time

# CPU-heavy task
def count(n):
    total = 0
    for i in range(n):
        total += i*i
    return total

N = 100_000_000  # large number for noticeable computation

start = time.time()

result = count(N)


print("without multi-threading took:", time.time() - start)

without multi-threading took: 3.2912373542785645


our expectation was: 4 threads → should be ~4x faster on 4 cores

why? -> GIL prevents threads from running Python bytecode in parallel

##### using multiprocessing in this case

In [11]:
from multiprocessing import Pool
import time

def count(n):
    total = 0
    for i in range(n):
        total += i*i
    return total

N = 100_000_000

with Pool(4) as p:  # 4 processes
    start = time.time()
    p.map(count, [N]*4)

print("Multiprocessing took:", time.time() - start)


Multiprocessing took: 5.525534391403198


---

### 4. Explain monkey patching in depth. Is it ever a good idea in production code? Provide an example of patching a standard library module and discuss risks like testability and maintainability.

monkey patching -> modify or extend the behavior of libraries, modules, or classes at runtime, without changing the original source code

Typically used for:

- Fixing bugs in third-party code.

- Adding features to existing libraries.

- Testing (mocking or stubbing functions).

In [12]:
# Original function
def greet(name):
    return f"Hello, {name}"

print(greet("Alice"))

Hello, Alice


In [13]:
# Monkey patch greet
def new_greet(name):
    return f"Hi there, {name}! Welcome."

greet = new_greet
print(greet("Alice"))

Hi there, Alice! Welcome.


**Monkey Patching Standard Library**

In [14]:
import datetime

# Original behavior
print(datetime.datetime.now())

2025-12-31 23:58:13.782621


In [15]:
# Monkey patch datetime.datetime.now
class PatchedDateTime(datetime.datetime):
    @classmethod
    def now(cls, tz=None):
        return cls(2000, 1, 1)

datetime.datetime = PatchedDateTime

print(datetime.datetime.now())

2000-01-01 00:00:00


Risks of Monkey Patching:

1. Maintainability

- Future updates to the library may break your patch.
- Other developers may not realize a function behaves differently.

2. Testability

- Tests might pass due to the patch but fail in production.
- If done at a global level, it can break unrelated modules.

4. Harder to Read

- Code behavior changes at runtime in a non-obvious way.
- New team members may be confused.

when to use?

- Testing/mocking: Patching functions for unit tests is common (unittest.mock.patch is safe and temporary).

- Hotfixing in legacy systems: If you cannot change the source code.

- Dynamic plugins: Sometimes frameworks use monkey patching internally.

Generally, avoid it in production code for core features because it introduces hidden complexity and fragility.

---

### 5. What happens internally when you use a decorator with arguments (e.g., @retry(max_attempts=3))? Implement a decorator that takes arguments and preserves the original function's metadata using functools.wraps.

consider:

```python
@retry(max_attempts=3)
def fetch_data():
    ...
```

This is syntactic sugar for:

```python
fetch_data = retry(max_attempts=3)(fetch_data)
```

what happens?

1. Python evaluates the decorator expression immediately ```retry(max_attempts=3)```
2. The returned decorator is called with the function ```decorator(fetch_data)``` -> Returns wrapper
3. Python rebinds the function name : ```fetch_data = wrapper```

In [1]:
import functools
import time

def retry(max_attempts=3, delay=0.0, exceptions=(Exception,)):
    """
    Decorator factory.
    """
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            attempts = 0
            while True:
                try:
                    return func(*args, **kwargs)
                except exceptions as e:
                    attempts += 1
                    if attempts >= max_attempts:
                        raise
                    if delay > 0:
                        time.sleep(delay)
        return wrapper
    return decorator


call stack visualization:

```python
fetch_data() ──▶ wrapper(*args)
                    │
                    ├─▶ try
                    │     └─▶ func(*args)
                    │
                    └─▶ except → retry logic

```

When you use a decorator, the original function is replaced by another function (wrapper).

So Python forgets who the original function was unless you explicitly tell it.

Without functools.wraps:

In [2]:
def my_decorator(func):
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper

@my_decorator
def fetch_data():
    """Fetch data from API"""
    pass

In [3]:
fetch_data.__name__

'wrapper'

Why?
Because fetch_data is no longer the original function.
It is now pointing to wrapper.

In [5]:
print(fetch_data.__doc__)

None


Because wrapper has no docstring, so the original docstring is lost.


So all metadata comes from wrapper, not the original function.

This breaks:

- Debugging

- Logging

- Stack traces

- Frameworks (FastAPI, Click, pytest)

- Documentation tools

- Introspection (inspect.signature, help())

In [8]:
# With functools.wraps

import functools

def my_decorator(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper


@my_decorator
def fetch_data():
    """Fetch data from API"""
    pass

In [9]:
fetch_data.__name__

'fetch_data'

With wraps(func):

- `__name__`

- `__doc__`

- `__annotations__`

- `__wrapped__` (important for introspection, debuggers, decorators stacking)

are copied correctly.

---

### 6. Differentiate between generators, coroutines (async def). Explain how asyncio uses coroutines for concurrency, and why generators are memory-efficient for large datasets.


#### 1. Generators

-> A generator is a function that yields values lazily using yield, rather than returning all values at once.

-> They implement the **iterator protocol**.

example:

```python
def count_up_to(n):
    i=0
    while i < n :
        yield i
        i += 1
```

key points:

- When you call `count_up_to(5)`, it returns a generator object, not the full list.
- You can iterate over it with `for` or `next()`.
- **Memory-efficient**: It produces one item at a time instead of holding the whole sequence in memory. Perfect for large datasets.

```python
gen = count_up_to(1_000_000_000)  # No memory crash
print(next(gen))  # 0
print(next(gen))  # 1
```

use cases : Processing streams, large files, or infinite sequences without loading everything into memory.

#### 2. Coroutines (`async def`)

- awaitable objects used in asynchronous programming

- non-blocking execution (enabling concurrency without threads)

In [6]:
import asyncio

async def task(name):
    print(f"{name} start")
    await asyncio.sleep(2)
    print(f"{name} end")

await asyncio.gather(task("A"), task("B"))

A start
B start
A end
B end


[None, None]

In [7]:
# without async

def task(name):
    print(f"{name} start")
    time.sleep(2)
    print(f"{name} end")

task("A")
task("B")

A start
A end
B start
B end


Concurrency: Multiple coroutines can be scheduled by the event loop using asyncio.create_task().

In [12]:
import asyncio

async def say_after(delay, message):
    await asyncio.sleep(delay)
    print(message)

async def main():
    # Schedule multiple coroutines concurrently
    task1 = asyncio.create_task(say_after(2, "Hello"))
    task2 = asyncio.create_task(say_after(1, "World"))
    
    print("Tasks scheduled")

    # Await both tasks to finish
    await task1
    await task2

await main()

Tasks scheduled
World
Hello


if we had just written:

```python
await say_after(2, "Hello")
await say_after(1, "World")
```

The calls would run sequentially, not concurrently.

Total time ≈ 3 seconds (2 + 1)


but create_task -> Schedules the coroutine to run concurrently in the event loop.

#### 3. How asyncio uses coroutines for concurrency?

- asyncio is built around an **event loop** and awaitable coroutines.

1. Coroutines are scheduled as **tasks**.
2. When a coroutine hits await `some_io()`, it pauses, and the event loop runs other coroutines.
3. Once `some_io()` completes, the coroutine resumes where it left off.
4. This allows thousands of I/O-bound tasks to run concurrently with minimal memory overhead.

In [15]:
import asyncio
import time

async def task(name, delay):
    await asyncio.sleep(delay)
    print(f"{name} done")

async def main():
    await asyncio.gather(
        task("A", 2),
        task("B", 1),
        task("C", 3),
    )
start = time.time()
await main()
print("Total time:", time.time() - start)

B done
A done
C done
Total time: 3.0018374919891357


- Single-threaded, non-blocking.

- Very memory-efficient compared to creating threads for each task.

#### 4. Why generators are memory-efficient for large datasets

- Generators compute values lazily, one at a time, rather than building a full list in memory.
- Only one value lives in memory at a time, even for infinite sequences.
- Avoids costly memory allocation for huge collections.

example:

```python
# Memory-heavy
nums = [x**2 for x in range(1_000_000_000)]  # -> ~8GB! : MemoryError likely

# Memory-efficient
nums_gen = (x**2 for x in range(1_000_000_000))  # Works fine -> nums_gen is a generator object.
```

In [16]:
nums_gen = (x**2 for x in range(1_000_000_000))  # Works fine

In [17]:
nums_gen

<generator object <genexpr> at 0x759ec6c98c70>

In [18]:
next(nums_gen)

0

In [19]:
next(nums_gen)

1

---